In [ ]:
Add your import statements and the database connection statements in the below code block

In [ ]:
import geopandas as gpd
import libpysal as lps
import numpy as np
import matplotlib.pyplot as plt
from esda.moran import Moran
from libpysal.weights import Rook

In [ ]:
Question 1:
Using the Libpysal Data, create a choropleth path of the United States that depicts the per capita income of each US state in 2009.

Note:

The Libpysal and GeoPandas libraries both contain utility functions that may make this task easier.
The choropleth map should have the following parameters:
legend
Title should be 'Per Capita Income of US States on Choropleth Map'
xlabel should be 'US Income'
Do not set font size or font-weight for titles and labels and figure size for the plot.

In [ ]:
# Load per capita income data
per_cap_data = lps.io.open(lps.examples.get_path("usjoin.csv"))

# Load US states shapefile
us_df = gpd.read_file(lps.examples.get_path("us48.shp"))

# Assign per capita income to dataframe
us_df['per_cap'] = np.array(per_cap_data.by_col("2009"))

# Plot the choropleth map
us_df.plot(column="per_cap", cmap="OrRd", legend=True, edgecolor="black")

# Add title and labels
plt.title("Per Capita Income of US States on Choropleth Map")
plt.xlabel("US Income")

# Show the plot
plt.show()

In [ ]:
Question 2:
Again using the Libpysal Data, create a proportional symbol map showing a dot at the centroid of each state that is scaled to the per capita income of each US state in 2009.

Note:

The demonstration notebook for this unit contains code that performs a similar task and may be a useful reference for your assignment.
The proportional symbol map should have the following parameters:
Title should be 'Proportional Symbol Map'
xlabel should be 'US Income'
Do not set font size or font-weight for titles and labels and figure size for the plot.

In [ ]:
# Compute centroids of each state
us_df['centroid'] = us_df.geometry.centroid

# Extract x and y coordinates of centroids
us_df['x'] = us_df.centroid.x
us_df['y'] = us_df.centroid.y

# Plot the base map
fig, ax = plt.subplots()
us_df.plot(ax=ax, color='lightgrey', edgecolor='black', legend=True)

# Plot proportional symbols
plt.scatter(us_df['x'], us_df['y'], s=us_df['per_cap'] / 500, color='red', alpha=0.6, edgecolors='black')

# Add title and labels
plt.title("Proportional Symbol Map")
plt.xlabel("US Income")

# Show the plot
plt.show()


In [ ]:
Question 3:
Using the same data, compute the value of Moran's I for the per capita income of each US state in 2009 using Rook Continuity. Report the value of I rounded to 4 decimal places (i.e. x.XXXX)

Note:

You need to merge the two datasets and then apply Rook Continuity
Use the esda library to calculate the Moran's value.
To calculate Moran’s I, you need two things: the value of interest (y) and a weight matrix (w).
Print only the calculated Moran's value rounded to 4 in the code cell below.
There should only be one print statement in the below cell.

In [ ]:
# Print available column names for debugging
# print(per_cap_data.header)

# Merge the datasets using the correct column name
income_df = gpd.GeoDataFrame({'STATE_NAME': per_cap_data.by_col("Name"),  # Corrected column name
                              'per_cap': np.array(per_cap_data.by_col("2009"))})

# Drop existing per_cap column if it exists to avoid merge conflicts
if "per_cap" in us_df.columns:
    us_df = us_df.drop(columns=["per_cap"])

us_df = us_df.merge(income_df, on="STATE_NAME", how="left")

# Compute Rook Continuity weight matrix
w = Rook.from_dataframe(us_df, use_index=True)
w.transform = "R"

# Compute Moran's I
moran = Moran(us_df["per_cap"].fillna(0), w)

# Print the Moran's I value rounded to 4 decimal places
print(round(moran.I, 4))